In [1]:
import sqlite3
import emoji
import re
from cleantext import clean

In [2]:
conn = sqlite3.connect("C:/Users/Afzal Sufiya/Documents/MasterThesis/AmazonReviews1.db")
cursor = conn.cursor()


In [3]:
# Fetch all rows from the table
cursor.execute("SELECT id, description FROM reviews")
rows = cursor.fetchall()


## Removing emojis from txt

In [69]:
# Function to remove emojis
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')


In [ ]:
for row in rows:
    id = row[0]
    description = row[1]
    new_desc = remove_emojis(description)
    cursor.execute("UPDATE reviews SET description = ? WHERE id = ?", (new_desc, id))
    

In [ ]:
conn.commit()


## Removing other noisy data

In [74]:
def cleaning_noise(description):
    # Clean the text while keeping basic punctuation, @, and $
    clean_noise = clean(description,
                                lower=False,            # Keep the original case
                                no_urls=True,           # Remove URLs
                                no_numbers=False,       # Keep numbers
                                no_digits=False,        # Keep digits
                                no_punct=False,         # Keep punctuation
                                no_currency_symbols=False, #keep currecny symbols
                                replace_with_punct="",  # Do not replace punctuation
                                replace_with_url="",
                                replace_with_number="",
                                replace_with_digit="",
                                replace_with_currency_symbol="",

                                additional_allowed_characters="@$")
    return clean_noise


In [75]:
# Update the desc column by removing emojis
for row in rows:
    id = row[0]
    description = row[1]
    new_desc = cleaning_noise(description)
    cursor.execute("UPDATE reviews SET description = ? WHERE id = ?", (new_desc, id))

TypeError: clean() got an unexpected keyword argument 'no_numbers'

In [42]:
# Commit the changes
conn.commit()


## translating non-English text into English

In [4]:
from langdetect import detect
from googletrans import Translator


In [5]:
# Function to detect language and translate non-English sentences to English
def translate_to_english(text):
    try:
        lang = detect(text)
        if lang == 'en':
            return text
        else:
            translator = Translator()
            translation = translator.translate(text, src=lang, dest='en')
            return translation.text
    except:
        return text
    

In [6]:
# Translate non-English sentences to English and update the 'desc' column
for id, description in rows:
    translated_desc = translate_to_english(description)
    cursor.execute("UPDATE reviews SET description = ? WHERE id = ?", (translated_desc, id))
    

In [4]:
conn.commit()
conn.close()

In [19]:
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect("C:/Users/Afzal Sufiya/Documents/MasterThesis/AmazonReviews.db")
cursor = conn.cursor()
# Execute the query to select 10 random rows
cursor.execute('SELECT * FROM reviews')

# Fetch the results
rows = cursor.fetchall()

# Get the column names
column_names = [description[0] for description in cursor.description]

# Create a DataFrame
df = pd.DataFrame(rows, columns=column_names)

#creating new feature "review length"
df['review_length']=df['description'].str.len()

# Descriptive statistics for the 'rating' column
rating_desc = df['review_length'].describe()
# Unique count for categorical columns
unique_counts = df.nunique()

print("Rating Statistics:")
print(rating_desc)
print("\nUnique Counts for Categorical Columns:")
print(unique_counts)

Rating Statistics:
count    51000.000000
mean       400.606588
std        681.154800
min          0.000000
25%         81.000000
50%        189.000000
75%        431.000000
max      24153.000000
Name: review_length, dtype: float64

Unique Counts for Categorical Columns:
product            499
subcategory         50
title            38168
rating               5
description      48825
Category            10
id               51000
review_length     2960
dtype: int64
